In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras, convert_to_tensor
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from scipy.optimize import minimize
from scipy.linalg import cholesky, LinAlgError
import matplotlib.pyplot as plt
import scipy
import time
from time import perf_counter


In [6]:
tf.keras.backend.set_floatx('float64') 
tf.keras.utils.set_random_seed(2) 

scaling = 1 
class CustomActivation(keras.layers.Layer):
    def __init__(self,**kwargs):
        super(CustomActivation, self).__init__()
        
    def call(self, x):
            return x*scaling
        
L = 2
x0 = -1
xf = x0 + L
tfinal = 1
L1 = 20
layer_dims = (2,L1,L1,L1,L1,1) 

initializer = tf.keras.initializers.VarianceScaling(
scale=1/scaling**2, mode='fan_avg', distribution='uniform')

X_input = Input((2,))
X = Dense(L1,activation="tanh")(X_input)
X = Dense(L1,activation="tanh")(X)
X = Dense(L1,activation="tanh")(X)
X = Dense(L1,activation="tanh")(X)
X = Dense(1,activation=CustomActivation(),kernel_initializer=initializer)(X)
N = Model(inputs=X_input, outputs=X)



def generate_inputs(Nint):
    t = tfinal*np.random.rand(Nint)
    x = L*np.random.rand(Nint) + x0
    X = np.hstack((t[:,None],x[:,None]))
    print('x',x.shape , 't', t.shape, 'X', X.shape)
    return convert_to_tensor(X)

def initial_points(N0): 
    
    t = np.zeros(N0)
    x = L*np.random.rand(N0) + x0
    X = np.hstack((t[:,None],x[:,None]))
    return convert_to_tensor(X)

def boundary_points(Nb): 
    t = tfinal*np.random.rand(Nb)
    x = L*np.random.randint(0,2,size=(Nb)) + x0
    X = np.hstack((t[:,None],x[:,None]))
    return convert_to_tensor(X)

def generate_validation(Nt,Nx): 
    x = np.linspace(x0,xf,Nx)
    t = np.linspace(0,tfinal,Nt)
    x,t = np.meshgrid(x,t)
    X = np.hstack((t.flatten()[:,None],x.flatten()[:,None]))
    print('x',x.shape , 't', t.shape, 'X', X.shape)
    return convert_to_tensor(X),t,x


def adaptive_rad(N,Nint,rad_args,Nsampling=50000):
    Xtest = generate_inputs(Nsampling)
    k1,k2 = rad_args
    Y = tf.math.abs(get_results(N,Xtest)[-1]).numpy()
    err_eq = np.power(Y,k1)/np.power(Y,k1).mean() + k2
    err_eq_normalized = (err_eq / sum(err_eq))
    X_ids = np.random.choice(a=len(Xtest), size=Nint, replace=False,
                         p=err_eq_normalized) 
    return tf.gather(Xtest,X_ids)

def uinit(X): #Initial condition
    x = X[:,1,None]
    return -tf.math.sin(np.pi*x)

def uleft(X):
    Xleft = X[X[:,1]==x0]
    t = Xleft[:,0]
    return tf.zeros([len(t),1],dtype=tf.float64)

def uright(X):
    Xright = X[X[:,1]==xf]
    t = Xright[:,0]
    return tf.zeros([len(t),1],dtype=tf.float64)


def output(N,X):
    Nout = N(X)
    u = Nout[:,0,None]
    return u

nu = 0.01/np.pi
def get_results(N,X):

   with tf.GradientTape(persistent=True, watch_accessed_variables=False) as gt1:
      gt1.watch(X)

      with tf.GradientTape(persistent=True, watch_accessed_variables=False) as gt2:
         gt2.watch(X)
         u = output(N,X)
      
      ugrad = gt2.gradient(u, X)
      u_t = ugrad[:,0]
      u_x = ugrad[:,1]
      
   u_xx = gt1.gradient(u_x, X)[:,1]
   fu = u_t + u[:,0]*u_x - nu*u_xx
   return u,fu


loss_function = keras.losses.MeanSquaredError() 
lam0 = 5 
lamB = 5 
def loss(fu,u0,u0pinn,ul,ulpinn,ur,urpinn):
    Ntot = fu.shape[0]
    zeros = tf.zeros([Ntot,1],dtype=tf.float64) 
    loss_value = loss_function(fu,zeros) + lam0*loss_function(u0,u0pinn) +\
                 lamB*(loss_function(ul,ulpinn) + loss_function(ur,urpinn))
    return loss_value


def grads(N,X,X0,Xb): 
    with tf.GradientTape() as tape2:
        _,fu = get_results(N,X)
        u0 = uinit(X0)
        u0pinn = output(N, X0)
        ul = uleft(Xb)
        ulpinn = output(N,Xb[Xb[:,1]==x0])
        ur = uright(Xb)
        urpinn = output(N,Xb[Xb[:,1]==xf])
        loss_value = loss(fu,u0,u0pinn,ul,ulpinn,ur,urpinn)
    gradsN = tape2.gradient(loss_value,N.trainable_variables)
    return gradsN,loss_value

@tf.function(jit_compile=True) 
def training(N,X,X0,Xb,optimizer): 
    parameter_gradients,loss_value = grads(N,X,X0,Xb)
    optimizer.apply_gradients(zip(parameter_gradients,N.trainable_variables))
    return loss_value


In [ ]:
Nepochs = 1000
Nchange = 500
Nint = 8000
N0 = 500
Nb = 500
Nprint = 100

k1 = 1
k2 = 1
rad_args = (k1, k2)

epochs_adam = np.arange(100, Nepochs + 100, 100)
loss_list = np.zeros(len(epochs_adam))
X = generate_inputs(Nint)
X0 = initial_points(N0)
Xb = boundary_points(Nb)

lr = tf.keras.optimizers.schedules.ExponentialDecay(5e-3, 1000, 0.98)
optimizer = Adam(lr, 0.99, 0.999, epsilon=1e-20)

start_time = perf_counter()
adam_epochs = np.arange(1, Nepochs + 1)
adam_loss_list = []

for i in range(Nepochs):
    if (i + 1) % Nchange == 0:
        X = adaptive_rad(N, Nint, rad_args)
        X0 = initial_points(N0)
        Xb = boundary_points(Nb)

    _, fu = get_results(N, X)
    u0 = uinit(X0)
    u0pinn = output(N, X0)
    ul = uleft(Xb)
    ulpinn = output(N, Xb[Xb[:, 1] == x0])
    ur = uright(Xb)
    urpinn = output(N, Xb[Xb[:, 1] == xf])
    loss_value = loss(fu, u0, u0pinn, ul, ulpinn, ur, urpinn)
    adam_loss_list.append(loss_value.numpy())

    training(N, X, X0, Xb, optimizer)

adam_end_time = perf_counter()
adam_training_time = adam_end_time - start_time
print(f"Total Adam training time: {adam_training_time:.2f} seconds")

with open("adam_training_time_burger.txt", "w") as f:
    f.write(f"Adam Training Time: {adam_training_time:.2f} seconds")


# --- Utilities for BFGS (no validation reporting) ---
def nested_tensor(grad, layer_dims, train_activations=False, bias=True):
    if train_activations == False:
        if bias:
            temp = [None] * (2 * len(layer_dims) - 2)
        else:
            temp = [None] * (2 * len(layer_dims) - 3)
        index = 0
        for i in range(len(temp)):
            if i % 2 == 0:
                expected_shape = (layer_dims[i // 2], layer_dims[i // 2 + 1])
                grad_slice_size = layer_dims[i // 2] * layer_dims[i // 2 + 1]
                temp[i] = np.reshape(
                    grad[index:index + layer_dims[i // 2] * layer_dims[i // 2 + 1]],
                    (layer_dims[i // 2], layer_dims[i // 2 + 1]),
                )
                index += layer_dims[i // 2] * layer_dims[i // 2 + 1]
            else:
                temp[i] = grad[index:index + layer_dims[i - i // 2]]
                index += layer_dims[i - i // 2]
        return temp
    else:
        temp = [None] * (3 * len(layer_dims) - 4)
        index = 0
        for i in range(len(temp)):
            if i % 3 == 0:
                temp[i] = np.reshape(
                    grad[index:index + layer_dims[i // 3] * layer_dims[i // 3 + 1]],
                    (layer_dims[i // 3], layer_dims[i // 3 + 1]),
                )
                index += layer_dims[i // 3] * layer_dims[i // 3 + 1]
            elif i % 3 == 1:
                temp[i] = grad[index:index + layer_dims[int((i + 2) / 3)]]
                index += layer_dims[int((i + 2) / 3)]
            else:
                temp[i] = grad[index]
                index += 1
        return temp


power = 1.0

@tf.function(jit_compile=True)
def loss_and_gradient_TF(N, X, X0, Xb):
    with tf.GradientTape() as tape:
        _, fu = get_results(N, X)
        u0 = uinit(X0)
        u0pinn = output(N, X0)
        ul = uleft(Xb)
        ulpinn = output(N, Xb[Xb[:, 1] == x0])
        ur = uright(Xb)
        urpinn = output(N, Xb[Xb[:, 1] == xf])
        loss_value = loss(fu, u0, u0pinn, ul, ulpinn, ur, urpinn) ** (1.0 / power)
    gradsN = tape.gradient(loss_value, N.trainable_variables)
    return loss_value, gradsN


def loss_and_gradient(weights, N, X, X0, Xb, layer_dims):
    resh_weights = nested_tensor(weights, layer_dims)
    N.set_weights(resh_weights)
    loss_value, grads = loss_and_gradient_TF(N, X, X0, Xb)
    grads_flat = np.concatenate([tf.reshape(g, [-1]).numpy() for g in grads])
    return loss_value.numpy(), grads_flat


Nbfgs = 50000
Nbatches = int(round(Nbfgs / Nchange))
epochs_bfgs = np.arange(0, Nbfgs + 100, 100)
epochs_bfgs += Nepochs
lossbfgs = np.zeros(len(epochs_bfgs))
Nprint = 100

initial_weights = np.concatenate([tf.reshape(w, [-1]).numpy() for w in N.weights])

cont = 0
time_ellapsed = np.array([])
initial_time = perf_counter()

def callback(*, intermediate_result):
    global cont, lossbfgs, Nprint
    if (cont + 1) % Nprint == 0 or cont == 0:
        loss_value = intermediate_result.fun ** power
        lossbfgs[(cont + 1) // Nprint] = loss_value
        print(f"Iteration {cont + 1}: Train Loss = {loss_value}")
    cont += 1


In [ ]:
H0 = tf.eye(len(initial_weights), dtype=tf.float64).numpy()
method = "BFGS"
method_bfgs = "SSBFGS_AB"
initial_scale = False

initial_time_bfgs = perf_counter()

while cont < Nbfgs:
    print(cont)
    result = minimize(
        loss_and_gradient,
        initial_weights,
        args=(N, X, X0, Xb, layer_dims),
        method=method,
        jac=True,
        options={
            "maxiter": Nchange,
            "gtol": 0,
            "hess_inv0": H0,
            "method_bfgs": method_bfgs,
            "initial_scale": initial_scale,
        },
        tol=0,
        callback=callback,
    )

    initial_weights = result.x
    H0 = result.hess_inv
    H0 = 0.5 * (H0 + H0.T)

    try:
        cholesky(H0)
    except LinAlgError:
        H0 = tf.eye(len(initial_weights), dtype=tf.float64).numpy()

    X = adaptive_rad(N, Nint, rad_args)
    X0 = initial_points(N0)
    Xb = boundary_points(Nb)

    _, fu = get_results(N, X)
    u0 = uinit(X0)
    u0pinn = output(N, X0)

    mask_left = tf.equal(Xb[:, 1], tf.cast(x0, Xb.dtype))
    mask_right = tf.equal(Xb[:, 1], tf.cast(xf, Xb.dtype))
    ul = uleft(Xb)
    ulpinn = output(N, tf.boolean_mask(Xb, mask_left))
    ur = uright(Xb)
    urpinn = output(N, tf.boolean_mask(Xb, mask_right))

    loss_value = loss(fu, u0, u0pinn, ul, ulpinn, ur, urpinn)
    initial_scale = False
